In [1]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.storagelevel import StorageLevel
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.sql.functions import when

In [2]:
data=[['Joe','Waterloo','9/21/19 3:04 AM','9/21/19 3:18 AM'],['Stacy','Kirkwood','8/4/19 3:06 PM','8/4/19 3:54 PM'],['John','Waterloo','9/21/19 8:48 AM','9/21/19 9:05 AM'],
      ['Stacy','Kirkwood','9/21/19 4:06 PM', '9/21/19 4:54 PM'],
     ['Mo','Hashmi','9/21/19 1:06 PM', '9/21/19 5:54 PM'],
     ['Murti','Hash','9/21/19 1:00 PM', '9/21/19 3:00 PM'],
     ['Floo','Shmi','9/21/19 9:10 PM', '9/21/19 11:54 PM']]
cSchema = StructType([StructField("User", StringType())\
                      ,StructField("Site", StringType())
                      , StructField("Sesh-Start", StringType())
                      , StructField("Sesh-End", StringType())])
df= spark.createDataFrame(data,schema=cSchema)
display(df)

User,Site,Sesh-Start,Sesh-End
Joe,Waterloo,9/21/19 3:04 AM,9/21/19 3:18 AM
Stacy,Kirkwood,8/4/19 3:06 PM,8/4/19 3:54 PM
John,Waterloo,9/21/19 8:48 AM,9/21/19 9:05 AM
Stacy,Kirkwood,9/21/19 4:06 PM,9/21/19 4:54 PM
Mo,Hashmi,9/21/19 1:06 PM,9/21/19 5:54 PM
Murti,Hash,9/21/19 1:00 PM,9/21/19 3:00 PM
Floo,Shmi,9/21/19 9:10 PM,9/21/19 11:54 PM


In [3]:
df1=df.withColumn("Start", F.from_unixtime(F.unix_timestamp("Sesh-Start",'MM/dd/yyyy hh:mm aa'),'20yy-MM-dd HH:mm:ss').cast("timestamp")).withColumn("End", F.from_unixtime(F.unix_timestamp("Sesh-End",'MM/dd/yyyy hh:mm aa'),'20yy-MM-dd HH:mm:ss').cast("timestamp")).drop("Sesh-Start","Sesh-End")
df1.show()


+-----+--------+-------------------+-------------------+
 User| Site| Start| End|
+-----+--------+-------------------+-------------------+
 Joe|Waterloo|2019-09-21 03:04:00|2019-09-21 03:18:00|
Stacy|Kirkwood|2019-08-04 15:06:00|2019-08-04 15:54:00|
 John|Waterloo|2019-09-21 08:48:00|2019-09-21 09:05:00|
Stacy|Kirkwood|2019-09-21 16:06:00|2019-09-21 16:54:00|
 Mo| Hashmi|2019-09-21 13:06:00|2019-09-21 17:54:00|
Murti| Hash|2019-09-21 13:00:00|2019-09-21 15:00:00|
 Floo| Shmi|2019-09-21 21:10:00|2019-09-21 23:54:00|
+-----+--------+-------------------+-------------------+

In [4]:
def yo(a,b):
  
  from datetime import datetime
  d1 = datetime.strptime(str(a), '%Y-%m-%d %H:%M:%S')
  d2 = datetime.strptime(str(b), '%Y-%m-%d %H:%M:%S')
  y=[]
  if d1.hour == d2.hour:
     y.append(d1.hour)
  else:
     for i in range(d1.hour,d2.hour+1):
        y.append(i)
        
  return y

rng= udf(yo, ArrayType(IntegerType()))
        
    

In [5]:
df2=df1.withColumn("new", rng(F.col("Start"),F.col("End"))).withColumn("new1",F.explode("new")).drop("new")

In [6]:
df2.show()

+-----+--------+-------------------+-------------------+----+
 User| Site| Start| End|new1|
+-----+--------+-------------------+-------------------+----+
 Joe|Waterloo|2019-09-21 03:04:00|2019-09-21 03:18:00| 3|
Stacy|Kirkwood|2019-08-04 15:06:00|2019-08-04 15:54:00| 15|
 John|Waterloo|2019-09-21 08:48:00|2019-09-21 09:05:00| 8|
 John|Waterloo|2019-09-21 08:48:00|2019-09-21 09:05:00| 9|
Stacy|Kirkwood|2019-09-21 16:06:00|2019-09-21 16:54:00| 16|
 Mo| Hashmi|2019-09-21 13:06:00|2019-09-21 17:54:00| 13|
 Mo| Hashmi|2019-09-21 13:06:00|2019-09-21 17:54:00| 14|
 Mo| Hashmi|2019-09-21 13:06:00|2019-09-21 17:54:00| 15|
 Mo| Hashmi|2019-09-21 13:06:00|2019-09-21 17:54:00| 16|
 Mo| Hashmi|2019-09-21 13:06:00|2019-09-21 17:54:00| 17|
Murti| Hash|2019-09-21 13:00:00|2019-09-21 15:00:00| 13|
Murti| Hash|2019-09-21 13:00:00|2019-09-21 15:00:00| 14|
Murti| Hash|2019-09-21 13:00:00|2019-09-21 15:00:00| 15|
 Floo| Shmi|2019-09-21 21:10:00|2019-09-21 23:54:00| 21|
 Floo| Shmi|2019-09-21 21:10:00|2019-09-21 23:54:00| 22|
 Floo| Shmi|2019-09-21 21:10:00|2019-09-21 23:54:00| 23|
+-----+--------+-------------------+-------------------+----+

In [7]:
df3=df2.withColumn("Seconds", when(F.hour("Start")==F.hour("End"), F.col("End").cast('long') - F.col("Start").cast('long'))
               .when(F.hour("Start")==F.col("new1"), 3600-F.minute("Start")*60)
               .when(F.hour("End")==F.col("new1"), F.minute("End")*60)
               .otherwise(3600))

In [8]:
df3.createOrReplaceTempView("final")

In [9]:
df3.show()

+-----+--------+-------------------+-------------------+----+-------+
 User| Site| Start| End|new1|Seconds|
+-----+--------+-------------------+-------------------+----+-------+
 Joe|Waterloo|2019-09-21 03:04:00|2019-09-21 03:18:00| 3| 840|
Stacy|Kirkwood|2019-08-04 15:06:00|2019-08-04 15:54:00| 15| 2880|
 John|Waterloo|2019-09-21 08:48:00|2019-09-21 09:05:00| 8| 720|
 John|Waterloo|2019-09-21 08:48:00|2019-09-21 09:05:00| 9| 300|
Stacy|Kirkwood|2019-09-21 16:06:00|2019-09-21 16:54:00| 16| 2880|
 Mo| Hashmi|2019-09-21 13:06:00|2019-09-21 17:54:00| 13| 3240|
 Mo| Hashmi|2019-09-21 13:06:00|2019-09-21 17:54:00| 14| 3600|
 Mo| Hashmi|2019-09-21 13:06:00|2019-09-21 17:54:00| 15| 3600|
 Mo| Hashmi|2019-09-21 13:06:00|2019-09-21 17:54:00| 16| 3600|
 Mo| Hashmi|2019-09-21 13:06:00|2019-09-21 17:54:00| 17| 3240|
Murti| Hash|2019-09-21 13:00:00|2019-09-21 15:00:00| 13| 3600|
Murti| Hash|2019-09-21 13:00:00|2019-09-21 15:00:00| 14| 3600|
Murti| Hash|2019-09-21 13:00:00|2019-09-21 15:00:00| 15| 0|
 Floo| Shmi|2019-09-21 21:10:00|2019-09-21 23:54:00| 21| 3000|
 Floo| Shmi|2019-09-21 21:10:00|2019-09-21 23:54:00| 22| 3600|
 Floo| Shmi|2019-09-21 21:10:00|2019-09-21 23:54:00| 23| 3240|
+-----+--------+-------------------+-------------------+----+-------+

In [10]:
display(spark.sql("Select new1, sum(Seconds) from final group by new1 order by new1"))

new1,sum(Seconds)
3,840
8,720
9,300
13,6840
14,7200
15,6480
16,6480
17,3240
21,3000
22,3600
